# Tugas Besar Data Visualization
## Covid-19 Daily Report

Anggota:
- Ghazi Ahmad Fadhlullah - 1301174062
- Iriyanto - 1301174295

In [ ]:
import numpy as np
import pandas as pd
import math
from datetime import datetime

import sys
import bokeh

from IPython import __version__ as ipython_version
from pandas import __version__ as pandas_version
from bokeh import __version__ as bokeh_version
print("IPython - %s" % ipython_version)
print("Pandas - %s" % pandas_version)
print("Bokeh - %s" % bokeh_version)

In [ ]:
# Standard imports 
from bokeh.io import output_notebook, output_file, show
from bokeh.plotting import figure
output_notebook()

In [ ]:
# Bokeh tools
from bokeh.models import ColumnDataSource, ColorBar, HoverTool, Legend
from bokeh.palettes import brewer
from bokeh.layouts import row, column, gridplot
from bokeh.models import CustomJS, Slider, Select, RadioGroup, Plot, Button, LinearAxis, Range1d, DatetimeTickFormatter
from bokeh.models.glyphs import Line, MultiLine

# Read datasets

In [ ]:
df_confirmed_global_raw = pd.read_csv('datasets/time_series_covid19_confirmed_global.csv')
df_confirmed_global_raw.head()

In [ ]:
df_recovered_global_raw = pd.read_csv('datasets/time_series_covid19_recovered_global.csv')
df_recovered_global_raw.head()

In [ ]:
df_deaths_global_raw = pd.read_csv('datasets/time_series_covid19_deaths_global.csv')
df_deaths_global_raw.head()

# Data Preparation

In [ ]:
def to_datetime(date):
    return datetime.strptime(date, '%m/%d/%y')

In [ ]:
df_confirmed_global = df_confirmed_global_raw.groupby('Country/Region').sum().drop(columns=['Lat', 'Long'])
df_confirmed_global = df_confirmed_global.T.reset_index().rename(columns={'index':'date'})
df_confirmed_global['date'] = df_confirmed_global['date'].apply(lambda x: to_datetime(x))
df_confirmed_global.tail()

In [ ]:
df_confirmed_temp = df_confirmed_global.drop(columns=['date'])
df_confirmed_diff = df_confirmed_temp.diff().fillna(0).astype('int32')
df_confirmed_diff[df_confirmed_diff < 0] = 0
df_confirmed_diff = df_confirmed_diff.reset_index().rename(columns={'index':'date'})
df_confirmed_diff['date'] = df_confirmed_global['date']
df_confirmed_diff.tail()

In [ ]:
df_recovered_global = df_recovered_global_raw.groupby('Country/Region').sum().drop(columns=['Lat', 'Long'])
df_recovered_global = df_recovered_global.T.reset_index().rename(columns={'index':'date'})
df_recovered_global['date'] = df_recovered_global['date'].apply(lambda x: to_datetime(x))
df_recovered_global.tail()

In [ ]:
df_recovered_temp = df_recovered_global.drop(columns=['date'])
df_recovered_diff = df_recovered_temp.diff().fillna(0).astype('int32')
df_recovered_diff[df_recovered_diff < 0] = 0
df_recovered_diff = df_recovered_diff.reset_index().rename(columns={'index':'date'})
df_recovered_diff['date'] = df_recovered_global['date']
df_recovered_diff.tail()

In [ ]:
df_deaths_global = df_deaths_global_raw.groupby('Country/Region').sum().drop(columns=['Lat', 'Long'])
df_deaths_global = df_deaths_global.T.reset_index().rename(columns={'index':'date'})
df_deaths_global['date'] = df_deaths_global['date'].apply(lambda x: to_datetime(x))
df_deaths_global.tail()

In [ ]:
df_deaths_temp = df_deaths_global.drop(columns=['date'])
df_deaths_diff = df_deaths_temp.diff().fillna(0).astype('int32')
df_deaths_diff[df_deaths_diff < 0] = 0
df_deaths_diff = df_deaths_diff.reset_index().rename(columns={'index':'date'})
df_deaths_diff['date'] = df_deaths_global['date']
df_deaths_diff.tail()

In [ ]:
df_global = df_confirmed_global.copy()
df_global = df_global[['date', 'Afghanistan']].drop('Afghanistan', axis=1)
df_global['confirmed'] = df_confirmed_global.iloc[:,1:].sum(axis=1)
df_global['recovered'] = df_recovered_global.iloc[:,1:].sum(axis=1)
df_global['deaths'] = df_deaths_global.iloc[:,1:].sum(axis=1)
df_global.head()

In [ ]:
df_global_diff = df_confirmed_global.copy()
df_global_diff = df_global_diff[['date', 'Afghanistan']].drop('Afghanistan', axis=1)
df_global_diff['confirmed'] = df_confirmed_diff.iloc[:,1:].sum(axis=1)
df_global_diff['recovered'] = df_recovered_diff.iloc[:,1:].sum(axis=1)
df_global_diff['deaths'] = df_deaths_diff.iloc[:,1:].sum(axis=1)
df_global_diff.head()

# Plot Global (Daily Total)

In [ ]:
def plot_global():
    source = ColumnDataSource(df_global)
    
    query = ['date', 'confirmed']
    data_filter = df_global[query].rename(columns={'confirmed' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Total Case",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Total Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(y[y.length - 1]+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    options = ['confirmed', 'recovered', 'deaths']
    
    # define select
    select = Select(title="Options:", value='confirmed', options=options)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    output_file('Global (Daily Total).html')
    show(layout)
    
plot_global()

# Plot Confirmed Global by Country (Daily Total)

In [ ]:
def plot_confirmed_global():
    source = ColumnDataSource(df_confirmed_global)
    
    query = ['date', 'Indonesia']
    data_filter = df_confirmed_global[query].rename(columns={'Indonesia' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Confirmed Case",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Total Confirmed Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(y[y.length - 1]+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    countries = df_confirmed_global_raw['Country/Region'].unique().tolist()
    
    # define select
    select = Select(title="Confirmed Case:", value='Indonesia', options=countries)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    output_file('Confirmed Global by Country (Daily Total).html')
    show(layout)
    
plot_confirmed_global()

# Plot Recovered Global by Country (Daily Total)

In [ ]:
def plot_recovered_global():
    source = ColumnDataSource(df_recovered_global)
    
    query = ['date', 'Indonesia']
    data_filter = df_recovered_global[query].rename(columns={'Indonesia' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Recovered Global",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Total Confirmed Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(y[y.length - 1]+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    countries = df_confirmed_global_raw['Country/Region'].unique().tolist()
    
    # define select
    select = Select(title="Recovered Global:", value='Indonesia', options=countries)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    output_file('Recovered Global by Country (Daily Total).html')
    show(layout)
    
plot_recovered_global()

# Plot Deaths Global by Country (Daily Total)

In [ ]:
def plot_deaths_global():
    source = ColumnDataSource(df_deaths_global)
    
    query = ['date', 'Indonesia']
    data_filter = df_deaths_global[query].rename(columns={'Indonesia' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Deaths Global",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Total Confirmed Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(y[y.length - 1]+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    countries = df_confirmed_global_raw['Country/Region'].unique().tolist()
    
    # define select
    select = Select(title="Deaths Global:", value='Indonesia', options=countries)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    output_file('Deaths Global by Country (Daily Total).html')
    show(layout)
    
plot_deaths_global()

# Plot Global (Daily Additional)

In [ ]:
def plot_global_diff():
    source = ColumnDataSource(df_global_diff)
    
    query = ['date', 'confirmed']
    data_filter = df_global_diff[query].rename(columns={'confirmed' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Daily Total Case",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+100)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Daily Total Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(Math.max.apply(Math, y)+100);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range, ax=p.yaxis), code=code)

    # country list
    options = ['confirmed', 'recovered', 'deaths']
    
    # define select
    select = Select(title="Options:", value='confirmed', options=options)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    output_file('Global (Daily Additional).html')
    show(layout)
    
plot_global_diff()

# Plot Confirmed Global by Country (Daily Additional)

In [ ]:
def plot_confirmed_global_diff():
    source = ColumnDataSource(df_confirmed_diff)
    
    query = ['date', 'Indonesia']
    data_filter = df_confirmed_diff[query].rename(columns={'Indonesia' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Daily Confirmed Case",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Daily Confirmed Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(Math.max.apply(Math, y)+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    countries = df_confirmed_global_raw['Country/Region'].unique().tolist()
    
    # define select
    select = Select(title="Confirmed Case:", value='Indonesia', options=countries)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    output_file('Confirmed Global by Country (Daily Additional).html')
    show(layout)
    
plot_confirmed_global_diff()

# Plot Recovered Global by Country (Daily Additional)

In [ ]:
def plot_recovered_global_diff():
    source = ColumnDataSource(df_recovered_diff)
    
    query = ['date', 'Indonesia']
    data_filter = df_recovered_diff[query].rename(columns={'Indonesia' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Daily Recovered Case",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Daily Recovered Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(Math.max.apply(Math, y)+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    countries = df_confirmed_global_raw['Country/Region'].unique().tolist()
    
    # define select
    select = Select(title="Recovered Case:", value='Indonesia', options=countries)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    output_file('Recovered Global by Country (Daily Additional).html')
    show(layout)
    
plot_recovered_global_diff()

# Plot Deaths Global by Country (Daily Additional)

In [ ]:
def plot_deaths_global_diff():
    source = ColumnDataSource(df_deaths_diff)
    
    query = ['date', 'Indonesia']
    data_filter = df_deaths_diff[query].rename(columns={'Indonesia' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Daily Deaths Case",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Daily Deaths Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(Math.max.apply(Math, y)+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    countries = df_confirmed_global_raw['Country/Region'].unique().tolist()
    
    # define select
    select = Select(title="Deaths Case:", value='Indonesia', options=countries)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    output_file('Deaths Global by Country (Daily Additional).html')
    show(layout)
    
plot_deaths_global_diff()